In [ ]:
import paho.mqtt.client as mqtt
import time
import threading

# MQTT settings
BROKER = "158.39.162.129"
PORT = 1883

ROLE = "leader"  # Define this JetBot as the leader
LEADER_IP = "192.168.1.12"  # Default IP for leader

if ROLE == "leader":
    print(f"This JetBot is the Leader with IP: {LEADER_IP}")
else:
    print("Error: Role not defined properly")
    exit()

TOPIC_LEADER = "platoon/leader"
TOPIC_FOLLOWERS = "platoon/followers"
TOPIC_STATUS = "platoon/status"
TOPIC_SYNC = "platoon/sync"
TOPIC_HEARTBEAT = "platoon/heartbeat"

# Initialize the MQTT client
client = mqtt.Client()

# Callback when connected to the broker
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker successfully.")
        client.subscribe([(TOPIC_FOLLOWERS, 0), (TOPIC_STATUS, 0)])
    else:
        print(f"Connection failed with code {rc}")

# Callback when disconnected from the broker
def on_disconnect(client, userdata, rc):
    if rc != 0:
        print("Unexpected disconnection. Reconnecting...")
    else:
        print("Disconnected cleanly.")

# Callback for received messages
def on_message(client, userdata, msg):
    print(f"Received on {msg.topic}: {msg.payload.decode()}")

# Heartbeat thread to notify followers the leader is online
def send_heartbeat():
    try:
        while True:
            client.publish(TOPIC_HEARTBEAT, "leader_online")
            time.sleep(10)  # Adjust heartbeat interval as needed
    except Exception as e:
        print(f"Heartbeat error: {e}")

# Command and synchronization loop
def publish_command(command):
    try:
        client.publish(TOPIC_LEADER, command)
        print(f"Command '{command}' sent to followers.")

        # Send sync signals to followers
        # sync_signal = input("Send sync signal (e.g., 'sync', 'align') or leave blank: ")
        if sync_signal:
            client.publish(TOPIC_SYNC, sync_signal)
            print(f"Sync signal '{sync_signal}' sent to followers.")
    except Exception as e:
        print("Error publishing command")

# Setup MQTT client callbacks
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message

# Connect to the broker
try:
    client.connect(BROKER, PORT, keepalive=60)
except Exception as e:
    print(f"Error connecting to broker: {e}")
    exit()

# Start MQTT loop and heartbeat thread
client.loop_start()
heartbeat_thread = threading.Thread(target=send_heartbeat, daemon=True)
heartbeat_thread.start()
